# ETL World Bank

## Overview

[DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. Different languages: `%LANGUAGE` syntax: Python, Scala, SQL, and R are all supported.

ETL stands for Extract, Transform, Load.

World Bank Indicator Data - This data contains socio-economic indicators for countries around the world. A few example indicators include population, arable land, and central government debt.

World Bank Project Data - This data set contains information about World Bank project lending since 1947.

Outline of this notebook:

Extract data from different sources such as:
csv files
json files
APIs

Transform data:
combining data from different sources
data cleaning
data types
parsing dates
file encodings
missing data
duplicate data
dummy variables
remove outliers
scaling features
engineering features

Load:
Send the transformed data to a database

ETL Pipeline
Code an ETL pipeline

In [0]:
%python
spark.catalog.listFunctions()

Out[58]: [Function(name='!', catalog=None, namespace=None, description='! expr - Logical not.', className='org.apache.spark.sql.catalyst.expressions.Not', isTemporary=True),
 Function(name='!=', catalog=None, namespace=None, description='expr1 != expr2 - Returns true if `expr1` is not equal to `expr2`.', className=None, isTemporary=True),
 Function(name='%', catalog=None, namespace=None, description='expr1 % expr2 - Returns the remainder after `expr1`/`expr2`.', className='org.apache.spark.sql.catalyst.expressions.Remainder', isTemporary=True),
 Function(name='&', catalog=None, namespace=None, description='expr1 & expr2 - Returns the result of bitwise AND of `expr1` and `expr2`.', className='org.apache.spark.sql.catalyst.expressions.BitwiseAnd', isTemporary=True),
 Function(name='*', catalog=None, namespace=None, description='expr1 * expr2 - Returns `expr1`*`expr2`.', className='org.apache.spark.sql.catalyst.expressions.Multiply', isTemporary=True),
 Function(name='+', catalog=None, na

## Extraction

In [0]:
%python
# File location and type
file_location = "/FileStore/tables/population_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
skiprows = 4
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
pop_data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("skiprows", skiprows) \
  .load(file_location)



display(pop_data)

Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Aruba,ABW,"Population, total",SP.POP.TOTL,54211,55438,56225,56695,57032,57360,57715,58055,58386,58726,59063,59440,59840,60243,60528,60657,60586,60366,60103,59980,60096,60567,61345,62201,62836,63026,62644,61833,61079,61032,62149,64622,68235,72504,76700,80324,83200,85451,87277,89005,90853,92898,94992,97017,98737,100031,100832,101220,101353,101453,101669,102053,102577,103187,103795,104341,104822,105264
Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351,9166764,9345868,9533954,9731361,9938414,10152331,10372630,10604346,10854428,11126123,11417825,11721940,12027822,12321541,12590286,12840299,13067538,13237734,13306695,13248370,13053954,12749645,12389269,12047115,11783050,11601041,11502761,11540888,11777609,12249114,12993657,13981231,15095099,16172719,17099541,17822884,18381605,18863999,19403676,20093756,20966463,21979923,23064851,24118979,25070798,25893450,26616792,27294031,28004331,28803167,29708599,30696958,31731688,32758020,33736494,34656032,35530081
Angola,AGO,"Population, total",SP.POP.TOTL,5643182,5753024,5866061,5980417,6093321,6203299,6309770,6414995,6523791,6642632,6776381,6927269,7094834,7277960,7474338,7682479,7900997,8130988,8376147,8641521,8929900,9244507,9582156,9931562,10277321,10609042,10921037,11218268,11513968,11827237,12171441,12553446,12968345,13403734,13841301,14268994,14682284,15088981,15504318,15949766,16440924,16983266,17572649,18203369,18865716,19552542,20262399,20997687,21759420,22549547,23369131,24218565,25096150,25998340,26920466,27859305,28813463,29784193
Albania,ALB,"Population, total",SP.POP.TOTL,1608800,1659800,1711319,1762621,1814135,1864791,1914573,1965598,2022272,2081695,2135479,2187853,2243126,2296752,2350124,2404831,2458526,2513546,2566266,2617832,2671997,2726056,2784278,2843960,2904429,2964762,3022635,3083605,3142336,3227943,3286542,3266790,3247039,3227287,3207536,3187784,3168033,3148281,3128530,3108778,3089027,3060173,3051010,3039616,3026939,3011487,2992547,2970017,2947314,2927519,2913021,2905195,2900401,2895092,2889104,2880703,2876101,2873457
Andorra,AND,"Population, total",SP.POP.TOTL,13411,14375,15370,16412,17469,18549,19647,20758,21890,23058,24276,25559,26892,28232,29520,30705,31777,32771,33737,34818,36067,37500,39114,40867,42706,44600,46517,48455,50434,52448,54509,56671,58888,60971,62677,63850,64360,64327,64142,64370,65390,67341,70049,73182,76244,78867,80991,82683,83861,84462,84449,83751,82431,80788,79223,78014,77281,76965
Arab World,ARB,"Population, total",SP.POP.TOTL,92490932,95044497,97682294,100411076,103239902,106174988,109230593,112406932,115680165,119016542,122398374,125807419,129269375,132863416,136696761,140843298,145332378,150133054,155183724,160392488,165689490,171051950,176490084,182005827,187610756,193310301,199093767,204942549,210844771,216787402,224735446,230829868,235037179,241286091,247435930,255029671,260843462,266575075,272235146,277962869,283832016,289850357,296026575,302434519,309162029,316264728,323773264,331653797,339825483,348145094,356508908,364895878,373306993,381702086,390043028,398304960,406452690,414491886
United Arab Emirates,ARE,"Population, total",SP.POP.TOTL,92634,101078,112472,125566,138529,150362,160481,170283,183194,203820,235499,278808,332760,397174,471364,554324,646943,748117,852262,952040,1042384,1120900,1189545,1253060,1318478,1391052,1472218,1560718,1655849,1756043,1860174,1970026,2086639,2207405,2328686,2448820,2571020,2700010,2838145,2988162,3154925,3326032,3507232,3741932,4087931,4579562,5242032,6044067,6894278,7666393,8270684,8672475,8900453,9006263,9070867,9154302,9269612,9400145
Argentina,ARG,"Population, total",SP.POP.TOTL,20619075,20953077,21287682,21621840,21953929,22283390,22608748,22932203,2326

In [0]:
%scala
// File location and type
val file_location = "/FileStore/tables/projects_data.csv"
val file_type = "csv"

// CSV options
val infer_schema = "false"
val first_row_is_header = "true"
val delimiter = ","

// The applied options are for CSV files. For other file types, these will be ignored.
val proj_data_scala = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)



display(proj_data_scala)

id,regionname,countryname,prodline,lendinginstr,lendinginstrtype,envassesmentcategorycode,supplementprojectflg,productlinetype,projectstatusdisplay,status,project_name,boardapprovaldate,board_approval_month,closingdate,lendprojectcost,ibrdcommamt,idacommamt,totalamt,grantamt,borrower,impagency,url,projectdoc,majorsector_percent,sector1,sector2,sector3,sector4,sector5,sector,mjsector1,mjsector2,mjsector3,mjsector4,mjsector5,mjsector,theme1,theme2,theme3,theme4,theme5,theme,goal,financier,mjtheme1name,mjtheme2name,mjtheme3name,mjtheme4name,mjtheme5name,location,GeoLocID,GeoLocName,Latitude,Longitude,Country,_c56
P162228,Other,World;World,RE,Investment Project Financing,IN,C,N,L,Active,Active,Creating a Trade in Value-Added Database for Africa,2018-06-28T00:00:00Z,June,null,"500,000",0,0,0,"500,000",null,null,http://projects.worldbank.org/P162228?lang=en,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
P163962,Africa,Democratic Republic of the Congo;Democratic Republic of the Congo,PE,Investment Project Financing,IN,B,N,L,Active,Active,Productive Inclusion Project,2018-06-28T00:00:00Z,June,2023-12-31T00:00:00Z,"200,000,000",0,"200,000,000","200,000,000",0,null,null,http://projects.worldbank.org/P163962?lang=en,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
P167672,South Asia,People's Republic of Bangladesh;People's Republic of Bangladesh,PE,Investment Project Financing,IN,null,Y,L,Active,Active,Additional Financing for Health Sector Support Project,2018-06-28T00:00:00Z,June,null,"50,000,000",0,"58,330,000","58,330,000",0,null,null,http://projects.worldbank.org/P167672?lang=en,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
P158768,South Asia,Islamic Republic of Afghanistan;Islamic Republic of Afghanistan,PE,Investment Project Financing,IN,A,N,L,Active,Active,Public-Private Partnerships and Public Investment Advisory Project,2018-06-27T00:00:00Z,June,2023-06-28T00:00:00Z,"50,000,000",0,"20,000,000","20,000,000",0,IIST,IIST,http://projects.worldbank.org/P158768?lang=en,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,!$!0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
P161364,Africa,Federal Republic of Nigeria;Federal Republic of Nigeria,PE,Investment Project Financing,IN,B,N,L,Active,Active,Nigeria For Women Project,2018-06-27T00:00:00Z,June,2023-05-31T00:00:00Z,"100,000,000",0,"100,000,000","100,000,000",0,null,null,http://projects.worldbank.org/P161364?lang=en,null,null,Social Protection!$!63!$!SA,Other Industry; Trade and Services!$!25!$!YZ,Other Agriculture; Fishing and Forestry!$!2!$!AZ,Other Public Administration!$!10!$!BZ,null,Social Protection;Social Protection;Other Industry; Trade and Services;Other Agriculture; Fishing and Forestry;Other Public Administration,null,null,null,null,null,Social Protection;Social Protection;Industry; Trade and Services;Agriculture; Fishing and Forestry;Public Administration,!$!0,null,null,null,null,null,null,null,null,null,null,null,null,0002327546!$!Ogun State!$!7!$!3.58333!$!NG;0002328925!$!Niger State!$!10!$!6!$!NG;0002565340!$!Abia State!$!5.41667!$!7.5!$!NG;0002565343!$!Edo!$!6.5!$!6!$!NG;0002597363!$!Kebbi State!$!11.5!$!4!$!NG;0002597366!$!Taraba State!$!8!$!10.5!$!NG,0002327546;0002328925;0002565340;0002565343;0002597363;0002597366,Ogun State;Niger State;Abia State;Edo;Kebbi State;Taraba State,7;10;5.41667;6.5;11.5;8,3.58333;6;7.5;6;4;10.5,NG;NG;NG;NG;NG;NG,null
P161483,Middle East and North Africa,Republic of Tunisia;Republic of Tunisia,PE,Development Policy Lending,AD,null,N,L,Active,Active,Tunisia Investment; Competitiveness and Inclusion,2018-06-27T00:0

In [0]:
%scala
import scala.io.Source

// File location and type
val file_location = "/FileStore/tables/mystery.csv"
val file_type = "csv"

// CSV options
val delimiter = ","

// The applied options are for CSV files. For other file types, these will be ignored.
var gdp_data_scala = Source.fromFile("/FileStore/tables/mystery.csv")

println(gdp_data_scala)

at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at scala.io.Source$.fromFile(Source.scala:94)
	at scala.io.Source$.fromFile(Source.scala:79)
	at scala.io.Source$.fromFile(Source.scala:57)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-3295903985122710:11)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-3295903985122710:60)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw$$iw$$iw$$iw.<init>(command-3295903985122710:62)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw$$iw$$iw.<init>(command-3295903985122710:64)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw$$iw.<init>(command-3295903985122710:66)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$$iw.<init>(command-3295903985122710:68)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read.<init>(command-3295903985122710:70)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$.<init>(command-3295903985122710:74)
	at $line6092cb6e2da8498785c9b9afba637c9251.$read$.<clinit>(command-3295903985122710)
	at $line6092cb6e2da8498785c9b9afba637c9251.$eval$.$print$lzycompute(<notebook>:7)
	at $line6092cb6e2da8498785c9b9afba637c9251.$eval$.$print(<notebook>:6)
	at $line6092cb6e2da8498785c9b9afba637c9251.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:747)
	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1020)
	at scala.tools.nsc.interpreter.IMain.$anonfun$interpret$1(IMain.scala:568)
	at scala.reflect.internal.util.ScalaClassLoader.asContext(ScalaClassLoader.scala:36)
	at scala.reflect.internal.util.ScalaClassLoader.asContext$(ScalaClassLoader.scala:116)
	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:41)
	at scala.tools.nsc.interpreter.IMain.loadAndRunReq$1(IMain.scala:567)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:594)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:564)
	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:223)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal.$anonfun$repl$1(ScalaDriverLocal.scala:225)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:1117)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:1070)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:225)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$23(DriverLocal.scala:729)
	at com.databricks.unity.EmptyHandle$.runWith(UCSHandle.scala:124)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$20(DriverLocal.scala:712)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:398)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:147)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:396)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:393)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:441)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:426)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:62)
	at com.databricks.bac

In [0]:
%sql
-- File location and type
SET file_location = "/FileStore/tables/rural_population_percent.csv";
SET file_type = "csv";

-- -- CSV options
SET infer_schema = "false";
SET header = "true";
SET delimiter = ",";
SET skiprows = 4;

-- The applied options are for CSV files. For other file types, these will be ignored.
DROP TABLE IF EXISTS rural_popPercent_sql;

CREATE TABLE IF NOT EXISTS rural_popPercent_sql
USING CSV
LOCATION "/FileStore/tables/rural_population_percent.csv"
OPTIONS (skiprows 4, infer_schema "false", header "true", delimiter ",");


SELECT * FROM rural_popPercent_sql;
-- display(rural_pop_sql)

Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Aruba,ABW,Rural population (% of total population),SP.RUR.TOTL.ZS,49.224,49.239,49.254,49.27,49.285,49.3,49.315,49.33,49.346,49.361,49.376,49.391,49.407,49.422,49.437,49.452,49.468,49.483,49.498,49.513,49.528,49.544,49.559,49.574,49.589,49.605,49.62,49.635,49.65,49.665,49.681,49.696,50.002,50.412,50.823,51.233,51.644,52.054,52.464,52.873,53.283,53.661,54.028,54.394,54.76,55.125,55.489,55.853,56.217,56.579,56.941,57.302,57.636,57.942,58.221,58.472,58.696,58.893
Afghanistan,AFG,Rural population (% of total population),SP.RUR.TOTL.ZS,91.779,91.492,91.195,90.89,90.574,90.25,89.915,89.57,89.214,88.848,88.471,88.083,87.684,87.274,86.851,86.417,85.971,85.513,85.042,84.565,84.319,84.07,83.818,83.563,83.304,83.042,82.777,82.509,82.237,81.962,81.684,81.403,81.118,80.83,80.538,80.243,79.945,79.644,79.339,79.03,78.718,78.404,78.085,77.763,77.438,77.105,76.763,76.413,76.054,75.687,75.311,74.926,74.532,74.129,73.718,73.297,72.868,72.43
Angola,AGO,Rural population (% of total population),SP.RUR.TOTL.ZS,89.565,89.202,88.796,88.376,87.942,87.496,87.035,86.559,86.068,85.564,85.043,84.566,84.125,83.676,83.215,82.745,82.263,81.772,81.27,80.758,80.234,79.701,79.157,78.602,78.035,77.459,76.872,76.275,75.666,75.048,74.418,73.779,73.128,72.47,71.8,71.12,70.43,69.732,69.025,68.308,67.581,66.848,66.105,65.355,64.595,63.831,63.058,62.278,61.491,60.701,59.903,59.1,58.301,57.51,56.726,55.95,55.181,54.422
Albania,ALB,Rural population (% of total population),SP.RUR.TOTL.ZS,69.295,69.057,68.985,68.914,68.842,68.77,68.698,68.626,68.554,68.452,68.26,68.067,67.873,67.679,67.484,67.288,67.092,66.895,66.698,66.5,66.238,65.976,65.713,65.448,65.183,64.917,64.65,64.381,64.112,63.842,63.572,63.3,62.751,62.201,61.646,61.089,60.527,59.965,59.399,58.831,58.259,57.565,56.499,55.427,54.349,53.269,52.185,51.098,50.009,48.924,47.837,46.753,45.67,44.617,43.591,42.593,41.624,40.684
Andorra,AND,Rural population (% of total population),SP.RUR.TOTL.ZS,41.55,39.017,36.538,34.128,31.795,29.555,27.407,25.359,23.412,21.576,19.845,18.22,16.699,15.284,13.968,12.748,11.618,10.58,9.622,8.743,7.93600000000001,7.2,6.526,5.911,5.351,4.84099999999999,4.675,4.822,4.973,5.128,5.288,5.47,5.676,5.889,6.11,6.339,6.575,6.81999999999999,7.07299999999999,7.334,7.605,7.944,8.35899999999999,8.79300000000001,9.249,9.705,10.162,10.637,11.133,11.648,12.183,12.74,13.292,13.835,14.367,14.885,15.388,15.873
Arab World,ARB,Rural population (% of total population),SP.RUR.TOTL.ZS,68.7080259489438,67.9655795464518,67.219820401638,66.4509596779045,65.6529640544118,64.813430428756,64.0164698825905,63.330975522168,62.6338164200751,61.9267725112699,61.2067075846122,60.4994606050061,59.7694031561536,59.0493978781337,58.4032831964468,57.8001220995549,57.2346676653567,56.7186391098259,56.2167115975255,55.7146927387959,55.1834118440946,54.6283783495423,54.0652969740215,53.4699296716967,52.833492544921,52.2226190942458,51.621277140871,51.0878985021993,50.6341109130518,50.2285249334738,49.6819887841947,49.2996003641955,49.3092480160596,48.9437816777014,48.6889940269548,48.1998300392349,48.0229193805594,47.807020720947,47.5572539040495,47.2960859768972,47.0315222273868,46.7670233323604,46.4992692397397,46.2225640945206,45.9270381567848,45.5983119081683,45.2441814239115,44.8832953951165,44.5188821390714,44.1567688882785,43.8045300821796,43.4634880632743,43.1286493068936,42.8037309903515,42.4912603659871,42.1917897852038,41.9060714489403,41.6319671533594
United Arab Emirates,ARE,Rural population (% of total population),SP.RUR.TOTL.ZS,26.5,25.617,24.752,23.907,23.081,22.277,21.492,20.728,20.2,20.2,20.2,20.2,20.2,20.2,20.2,20.2,20.097,19.894,19.691,19.49,19.29,19.317,19.534,19.753,19.97

In [0]:
%r
library(SparkR)
# File location and type
file_location = "/FileStore/tables/gdp_data.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
header = "true"
delimiter = ","


# The applied options are for CSV files. For other file types, these will be ignore
gdp_data_r <- read.df(file_location, source = file_type, header = header, inferSchema = infer_schema)

head(gdp_data_r)




        Data Source World Development Indicators               _c2
1 Last Updated Date                   2018-06-28              <NA>
2      Country Name                 Country Code    Indicator Name
3             Aruba                          ABW GDP (current US$)
4       Afghanistan                          AFG GDP (current US$)
5            Angola                          AGO GDP (current US$)
6           Albania                          ALB GDP (current US$)

In [0]:
%python
pop_data.head(5)

Out[41]: [Row(_c0='Data Source', _c1='World Development Indicators', _c2=None),
 Row(_c0='Last Updated Date', _c1='2018-06-28', _c2=None),
 Row(_c0='Country Name', _c1='Country Code', _c2='Indicator Name'),
 Row(_c0='Aruba', _c1='ABW', _c2='Population, total'),
 Row(_c0='Afghanistan', _c1='AFG', _c2='Population, total')]

In [0]:
%r
head(pop_data)

Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'head': object 'pop_data' not found

In [0]:
pop_data.columns

Out[16]: ['_c0', '_c1', '_c2']

In [0]:
rural_ppop_dataop_data.dtypes

Out[15]: [('_c0', 'string'), ('_c1', 'string'), ('_c2', 'string')]

In [0]:
%python
# Create a view or table

temp_table_name = "mystery_csv"

rural_pop_data.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

SELECT * FROM mystery_csv
LIMIT 5

_c0,_c1,_c2
Data Source,World Development Indicators,null
Last Updated Date,2018-06-28,null
Country Name,Country Code,Indicator Name
Aruba,ABW,Rural population (% of total population)
Afghanistan,AFG,Rural population (% of total population)


In [0]:
%python
# This table will persist across cluster restarts as well as allow various users across different notebooks to query this data.


permanent_table_name = "mystery_csv"
df.write.format("parquet").saveAsTable(permanent_table_name)

# df.write.format("parquet").saveAsTable(permanent_table_name)

## Transformation